In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import json
import urllib
import requests
import json
import math
import os 
from tqdm import tqdm
import folium
import time

# 1. Geo Data Generating

In [2]:
geo_path = 'seoul_lines.json'
seoul_line=json.load(open(geo_path, encoding='utf-8'))
criterion = pd.read_csv('criterion.csv').iloc[:,1:]

In [3]:
si = []
gu = []
dong = []
line_jwapyo = []

for i in range(424) : 

    si_ = seoul_line['features'][i]['properties']['sidonm'] + ' '
    gu_ = seoul_line['features'][i]['properties']['sggnm'] + ' '
    dong_ = seoul_line['features'][i]['properties']['adm_nm']
    dong_ = dong_.replace(si_, '')
    dong_ = dong_.replace(gu_, '')
    si_ = si_.replace(' ', '')
    gu_ = gu_.replace(' ', '')
    si.append(si_)
    gu.append(gu_)
    dong.append(dong_)

    jwa = (seoul_line['features'][i]['geometry']['coordinates'][0][0])
    line_jwapyo.append(jwa)
    


geo_line = pd.DataFrame({'gu' : gu, 'dong' : dong, 'jwapyo' : line_jwapyo})
geo_line

,gu,dong,jwapyo
0,종로구,사직동,"[[126.97688884274817, 37.575650779448786], [12..."
1,종로구,삼청동,"[[126.98268938649305, 37.5950655194224], [126...."
2,종로구,부암동,"[[126.97585113775686, 37.59656422224409], [126..."
3,종로구,평창동,"[[126.97507466788086, 37.63138628651299], [126..."
4,종로구,무악동,"[[126.96067353273895, 37.58079784202973], [126..."
...,...,...,...
419,강동구,둔촌1동,"[[127.14580008867549, 37.521989966841396], [12..."
420,강동구,둔촌2동,"[[127.15355242225388, 37.533996519079544], [12..."
421,강동구,암사1동,"[[127.14447178485771, 37.55477055824149], [127..."
422,강동구,천호2동,"[[127.13457783289466, 37.54712046995205], [127..."


In [13]:
## 여기가 넣고싶은 건물데이터

building = pd.read_csv('서울교통공사_역주소및전화번호_20200715.csv', encoding = 'cp949').iloc[:, 1:]
building['address'] = building['역주소']

In [11]:
from integrating_model import integrating_model

model = integrating_model(seoul_line, geo_line, criterion)

자치구 행정동 좌표 붙이기는 model name : tagging
인접 시설물 찾기는 model name : nearest_imputation


In [15]:
building = building.fillna('NA')
building

,역번호,호선,역명,역주소,address
0,150,1,서울역,서울특별시 중구 세종대로 지하 2 (남대문로 5가),서울특별시 중구 세종대로 지하 2 (남대문로 5가)
1,151,1,시청,서울특별시 중구 세종대로 지하 101 (정동),서울특별시 중구 세종대로 지하 101 (정동)
2,152,1,종각,서울특별시 종로구 종로 지하 55 (종로1가),서울특별시 종로구 종로 지하 55 (종로1가)
3,153,1,종로3가,서울특별시 종로구 종로 지하 129 (종로3가),서울특별시 종로구 종로 지하 129 (종로3가)
4,154,1,종로5가,서울특별시 종로구 종로 지하 216 (종로5가),서울특별시 종로구 종로 지하 216 (종로5가)
...,...,...,...,...,...
255,NA,NA,신촌기차,서울 서대문구 신촌동,서울 서대문구 신촌동
256,NA,NA,서강대,서울 마포구 서강로 118,서울 마포구 서강로 118
257,NA,NA,서빙고,서울 용산구 서빙고동,서울 용산구 서빙고동
258,NA,NA,한남,서울 용산구 독서당로6길,서울 용산구 독서당로6길


In [16]:
first_step = model.tagging(building, 'f9f190ad1b7a28e6afa2b7b585738e48')

100%|████████████████████████████████████████████████████████████████████████████████| 260/260 [00:19<00:00, 13.59it/s]


In [19]:
first_step[first_step.si == 0]

,역번호,호선,역명,역주소,address,si,gu,dong,jwapyo
193,2722,7,상봉,서울특별시 중랑구 망우로 지하 297,서울특별시 중랑구 망우로 지하 297,0,0,0,"[0, 0]"
251,4137,9,둔촌오륜,서울특별시 송파구 강동대로 327,서울특별시 송파구 강동대로 327,0,0,0,"[0, 0]"
254,NA,NA,가좌,서울 서대문구 남가좌동 122-1,서울 서대문구 남가좌동 122-1,0,0,0,"[0, 0]"


In [44]:
first_step.iloc[193, 5] = '서울'
first_step.iloc[193, 6] = '중랑구'
first_step.iloc[193, 7] = '상봉1동'
first_step.iloc[193, 8] = ['127.0813674', '37.5968816']

first_step.iloc[251, 5] = '서울'
first_step.iloc[251, 6] = '강동구'
first_step.iloc[251, 7] = '둔촌1동'
first_step.iloc[251, 8] = ['127.1376551', '37.5253956']

first_step.iloc[254, 5] = '서울'
first_step.iloc[254, 6] = '마포구'
first_step.iloc[254, 7] = '망원2동'
first_step.iloc[254, 8] = ['126.9129333', '37.5682837']

In [48]:
a2

,역번호,호선,역명,역주소,address,si,gu,dong,jwapyo
0,150,1,서울역,서울특별시 중구 세종대로 지하 2 (남대문로 5가),서울특별시 중구 세종대로 지하 2 (남대문로 5가),서울,중구,회현동,"[126.972554606456, 37.5571594835988]"
1,151,1,시청,서울특별시 중구 세종대로 지하 101 (정동),서울특별시 중구 세종대로 지하 101 (정동),서울,중구,소공동,"[126.976983237511, 37.5654385176408]"
2,152,1,종각,서울특별시 종로구 종로 지하 55 (종로1가),서울특별시 종로구 종로 지하 55 (종로1가),서울,종로구,종로1.2.3.4가동,"[126.983239678933, 37.5702140381269]"
3,153,1,종로3가,서울특별시 종로구 종로 지하 129 (종로3가),서울특별시 종로구 종로 지하 129 (종로3가),서울,종로구,종로1.2.3.4가동,"[126.99202930223, 37.5704266025836]"
4,154,1,종로5가,서울특별시 종로구 종로 지하 216 (종로5가),서울특별시 종로구 종로 지하 216 (종로5가),서울,종로구,종로5.6가동,"[127.001916647342, 37.5709075371016]"
...,...,...,...,...,...,...,...,...,...
255,NA,NA,신촌기차,서울 서대문구 신촌동,서울 서대문구 신촌동,서울,서대문구,신촌동,"[126.93927687604, 37.5651616940519]"
256,NA,NA,서강대,서울 마포구 서강로 118,서울 마포구 서강로 118,서울,마포구,대흥동,"[126.934808313154, 37.5527051004675]"
257,NA,NA,서빙고,서울 용산구 서빙고동,서울 용산구 서빙고동,서울,용산구,서빙고동,"[126.994638088433, 37.5204623812777]"
258,NA,NA,한남,서울 용산구 독서당로6길,서울 용산구 독서당로6길,서울,용산구,한남동,"[127.008114489392, 37.5300173532496]"


In [50]:
## 여기서 데이터를 전처리해야함.
first_step.iloc[218, 7] = '오류1동'
a2 = first_step[first_step.si == '서울']

In [51]:
second_step = model.nearest_imputation(a2, 500)

  0%|                                                                                          | 0/241 [00:00<?, ?it/s]

입력된 시설물의 수는  241
정상적으로 카운팅된 수는  241.0


Nearest Imputation is on process..


100%|████████████████████████████████████████████████████████████████████████████████| 241/241 [03:25<00:00,  1.17it/s]


In [58]:
subway = second_step

# 2. Integrating the Final Data

In [68]:
medical = pd.read_csv('의료시설(최종).csv', encoding= 'cp949').iloc[:, 1:]
facility = pd.read_csv('장애인시설(최종).csv', encoding= 'cp949').iloc[:, 1:]
sleep = pd.read_csv('관광숙박(최종).csv', encoding= 'cp949').iloc[:, 1:]
eat = pd.read_csv('관광음식점(최종).csv', encoding= 'cp949').iloc[:, 1:]
library = pd.read_csv('도서관(최종).csv', encoding= 'cp949').iloc[:, 1:]

In [74]:
medical.index = medical.dong
subway.index = subway.dong
facility.index = facility.dong
sleep.index = sleep.dong
eat.index = eat.dong
library.index =library.dong

In [76]:
medical['medical_facility'] = medical['actually_there'] + medical['probably there']
subway['metro'] = subway['actually_there'] + subway['probably there']
facility['welfare_facility'] = facility['actually_there'] + facility['probably there']
sleep['motel'] = sleep['actually_there'] + sleep['probably there']
eat['restaurant'] = eat['actually_there'] + eat['probably there']
library['library'] = library['actually_there'] + library['probably there']

In [244]:
final_data = pd.concat([integrated_data['number_of_disabled'], facility['welfare_facility'], medical['medical_facility'],
            library['library'], eat['restaurant'],
            sleep['motel'], subway['metro']], axis = 1)

In [245]:
final_data

,number_of_disabled,welfare_facility,medical_facility,library,restaurant,motel,metro
dong,,,,,,,
사직동,295,4.0,2.0,33.0,5.0,3.0,4.0
삼청동,112,1.0,0.0,17.0,1.0,7.0,1.0
부암동,316,0.0,0.0,4.0,3.0,1.0,0.0
평창동,538,0.0,0.0,5.0,1.0,1.0,0.0
무악동,337,1.0,0.0,9.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...
둔촌1동,3,1.0,1.0,4.0,3.0,0.0,3.0
둔촌2동,960,2.0,8.0,11.0,4.0,6.0,5.0
암사1동,1688,4.0,3.0,11.0,1.0,0.0,2.0


In [155]:
people = pd.read_csv('서울시 장애인 현황 (동별 나이별).txt', sep = '	')
people = people.iloc[3:]
people = people[people.동 != '소계']
people = people[people.동 != '기타']
handi_people = people.iloc[:, 1:4]

In [156]:
handi_people['dong'] = handi_people.동
handi_people.index = handi_people.dong
handi_people['people'] = handi_people['합계']

C:\Users\kswoo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\kswoo\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [185]:
integrated_data['key_'] = integrated_data.index
handi_people['key_'] = handi_people.index

C:\Users\kswoo\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [204]:
c = pd.merge(integrated_data, handi_people,  
        on = 'key_')
sibal_ = c.key_

In [224]:
print(integrated_data[integrated_data.index != handi_people.index].index)
print(handi_people[integrated_data.index != handi_people.index].index)

Index(['답십리2동', '장안1동', '장안2동', '이문1동', '이문2동', '답십리1동', '잠실본동', '잠실4동',
       '잠실6동', '잠실7동', '잠실2동', '잠실3동', '장지동', '위례동'],
      dtype='object', name='dong')
Index(['답십리1동', '답십리2동', '장안1동', '장안2동', '이문1동', '이문2동', '장지동', '위례동', '잠실본동',
       '잠실4동', '잠실6동', '잠실7동', '잠실2동', '잠실3동'],
      dtype='object', name='dong')


In [237]:
people_ = []
for i in tqdm(range(424)) : 
    people_.append(int(handi_people[handi_people.index == integrated_data.index[i]].people.values[0]))

100%|██████████████████████████████████████████████████████████████████████████████| 424/424 [00:00<00:00, 3864.82it/s]


In [240]:
integrated_data['number_of_disabled'] = people_

In [246]:
come = []
for k in range(424) : 
    if handi_people.index[k] in integrated_data.index : 
        None
    else : 
        come.append(k)
        
come2 = []
for k in range(424) : 
    if integrated_data.index[k] in handi_people.index : 
        None
    else : 
        come2.append(k)

In [252]:
taxi_data= pd.read_csv('final_taxi_data.csv').iloc[:, 1:]
taxi_data

,total_date,car_number,car_type,receipttime,settime,ridetime,startpos1,endpos1,startpos2_,endpos2_,start_location,end_location,gu_move
0,20190101,5815,중형승합,2019-01-01 오전 12:02:00,2019-01-01 오전 3:56:24,2019-01-01 오전 4:15:34,종로구,성북구,삼청동,월곡1동,1,1,2
1,20190101,5815,중형승합,2019-01-01 오전 12:13:00,2019-01-01 오전 3:17:56,2019-01-01 오전 3:36:11,성북구,강북구,보문동,삼각산동,1,1,2
2,20190101,1313,중형승합,2019-01-01 오전 12:15:00,2019-01-01 오전 3:03:30,2019-01-01 오전 3:23:48,강북구,노원구,인수동,하계2동,1,1,2
3,20190101,1601,중형승합,2019-01-01 오전 12:21:00,2019-01-01 오전 1:36:00,2019-01-01 오전 2:13:11,은평구,은평구,구산동,역촌동,1,1,2
4,20190101,1373,중형승합,2019-01-01 오전 12:30:00,2019-01-01 오전 3:45:19,2019-01-01 오전 4:13:13,영등포구,관악구,여의동,난곡동,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109036,20191231,7926,중형승합,2019-12-31 오후 11:23:00,2020-01-01 오전 1:56:21,2020-01-01 오전 2:18:07,영등포구,은평구,여의동,증산동,0,1,1
1109037,20191231,7922,중형승합,2019-12-31 오후 11:26:00,2020-01-01 오전 4:43:42,2020-01-01 오전 5:10:30,금천구,영등포구,시흥2동,여의동,0,0,0
1109038,20191231,7922,중형승합,2019-12-31 오후 11:27:00,2020-01-01 오전 3:43:51,2020-01-01 오전 4:06:33,은평구,강서구,역촌동,가양2동,1,0,1
1109039,20191231,7917,중형승합,2019-12-31 오후 11:31:00,2020-01-01 오전 2:35:08,2020-01-01 오전 2:56:38,강북구,노원구,삼양동,중계2.3동,1,1,2


In [254]:
departure = taxi_data.groupby(by = ['startpos2_'], as_index = False).count()
arrival = taxi_data.groupby(by = ['endpos2_'], as_index = False).count()

In [284]:
departure_ = []
for i in tqdm(range(424)) : 
    daf = departure[departure.startpos2_ == final_data.index[i]].total_date.values
    if daf.shape[0] == 0 : 
        departure_.append(0)
    else : 
        departure_.append(daf[0])
    
arrival_ = []
for s in tqdm(range(424)) : 
    daf = arrival[arrival.endpos2_ == final_data.index[s]].total_date.values
    if daf.shape[0] == 0: 
        arrival_.append(0)
    else : 
        arrival_.append(daf[0])

100%|██████████████████████████████████████████████████████████████████████████████| 424/424 [00:00<00:00, 2742.94it/s]


In [285]:
final_data['taxi_departure'] = departure_
final_data['taxi_arrival'] = arrival_

In [287]:
final_data

,number_of_disabled,welfare_facility,medical_facility,library,restaurant,motel,metro,taxi_departure,taxi_arrival
dong,,,,,,,,,
사직동,295,4.0,2.0,33.0,5.0,3.0,4.0,2345,2777
삼청동,112,1.0,0.0,17.0,1.0,7.0,1.0,274,306
부암동,316,0.0,0.0,4.0,3.0,1.0,0.0,1135,1128
평창동,538,0.0,0.0,5.0,1.0,1.0,0.0,1291,1384
무악동,337,1.0,0.0,9.0,0.0,0.0,2.0,507,532
...,...,...,...,...,...,...,...,...,...
둔촌1동,3,1.0,1.0,4.0,3.0,0.0,3.0,68,72
둔촌2동,960,2.0,8.0,11.0,4.0,6.0,5.0,6319,6899
암사1동,1688,4.0,3.0,11.0,1.0,0.0,2.0,2387,2015


In [304]:
seoul_ = np.repeat('서울시', 424)
final = pd.concat([pd.DataFrame({'si' : seoul_}), 
                   medical.iloc[:, :2].reset_index(drop = True), 
                   final_data.reset_index(drop = True)], axis = 1)

In [305]:
final.to_csv('서울시_행정동_장애인_통합_데이터.csv')

,si,gu,dong,number_of_disabled,welfare_facility,medical_facility,library,restaurant,motel,metro,taxi_departure,taxi_arrival
0,서울시,종로구,사직동,295,4.0,2.0,33.0,5.0,3.0,4.0,2345,2777
1,서울시,종로구,삼청동,112,1.0,0.0,17.0,1.0,7.0,1.0,274,306
2,서울시,종로구,부암동,316,0.0,0.0,4.0,3.0,1.0,0.0,1135,1128
3,서울시,종로구,평창동,538,0.0,0.0,5.0,1.0,1.0,0.0,1291,1384
4,서울시,종로구,무악동,337,1.0,0.0,9.0,0.0,0.0,2.0,507,532
...,...,...,...,...,...,...,...,...,...,...,...,...
419,서울시,강동구,둔촌1동,3,1.0,1.0,4.0,3.0,0.0,3.0,68,72
420,서울시,강동구,둔촌2동,960,2.0,8.0,11.0,4.0,6.0,5.0,6319,6899
421,서울시,강동구,암사1동,1688,4.0,3.0,11.0,1.0,0.0,2.0,2387,2015
422,서울시,강동구,천호2동,1765,2.0,7.0,9.0,0.0,8.0,2.0,4840,5387
